In [ ]:
from __future__ import division
from sympy import *
import numpy as np

# Containers
x0 = np.random.randint(1,100, size=2)
eps = 0.001
k=0
x, y, lamda = sympy.symbols('x y lamda')
xi=[]
direction=[]


# True Functions
"""
f = (lambda x,y: x**2 + y**2)
gradient_f = lambda x,y : np.array([2*x,2*y])
hessian_f = lambda x,y: np.matrix([[2,0],
                                 [0,2]])

# Symbolic Functions
get_fsym_gradient = lambda x,y: [fsym.diff(var) for var in (x,y)]
fsym = (x**2)+(y**2) #worked
"""

f = (lambda x,y : (1/2)*x**2+x*y+(1/2)*y**2)
gradient_f = (lambda x,y :np.array([x+y, y+x]))
hessian_f = (lambda x,y :np.matrix([[1,1],
                                    [1,1]]))
fsym = (1/2)*x**2+x*y+(1/2)*y**2
get_fsym_gradient = lambda x,y: [fsym.diff(var) for var in (x,y)]

# Start
xi.append(x0)

print ("Create hyperparameters ...  \n")
print ("Initial Position:", xi[k])
print ("Threshold:", epsilon)
print ("Symbols:", x,y,lamda)
print ("fsym:", fsym)
print ("gradient_fsym:", get_fsym_gradient(x,y))
print ("Execute Procedure ...  \n")


def golden_search(fun,low=0,high=1,eps=0.1, verbose=False):

    if verbose:
        print("*******************************")
        print("Executing Golden Ratio Search")
        print("*******************************")
        
        
    
    # Define Constants
    alpha = 0.618
    k=0
    
    # Create lists
    lam=list()
    mu=list()
    a=list()
    b=list()
    distance=list()
    
    # Init
    a.append(low)
    b.append(high)

    lam0 = a[k] + (1-alpha)*(b[k]-a[k])
    mu0 = a[k] + alpha*(b[k]-a[k])
    
    lam.append(lam0)
    mu.append(mu0)
    
    if verbose:
        print("************************")
        print(("[a,b]"), "|b[k]-a[k]|")
        print("************************")
        
    while True:
        distance.append(abs(b[k]-a[k])) 
        if verbose:
            print(a[k],b[k], distance[k])
        if(distance[k] < eps): # optimal soln lies within [a,b]
            break
        elif fun(lam[k]) > fun(mu[k]):
            # Step 2
            a.append(lam[k])
            b.append(b[k])
            lam.append(mu[k])
            mu.append(a[k+1] + alpha*(b[k+1]-a[k+1]))
        elif fun(lam[k]) <= fun(mu[k]):
            # Step3
            a.append(a[k])
            b.append(mu[k])
            mu.append(lam[k])
            lam.append(a[k+1]+(1-alpha)*(b[k+1]-a[k+1]))
        else:
            print("Something went wrong.")
            
        k += 1
    
    return (a[-1] + b[-1]) /2

def get_xi_of_lambda(xold, d, verbose=False):
    L=xold - d*lamda
    xi_lambda = fsym.subs({x:L[0], y:L[1]})
    
    f_lambda = sympy.lambdify([lamda], xi_lambda, modules=['math'])
    
    if verbose==True:
        print("Lambda Function = ", xi_lambda)
    return f_lambda

In [156]:
print("Starting Position = ", xi[k])
while True:
    print("Objective Value = ", f(xi[k][0],xi[k][1]))
    
    direction.append(gradient_f(xi[k][0], xi[k][1]) )
    f_lambda = get_xi_of_lambda(xi[k], direction[k], verbose=True)
    lam = golden_search(f_lambda, low=0, high=1, eps=0.01, verbose=True)
    xi.append(xi[k]-lam*gradient_f(xi[k][0], xi[k][1]))
    print("Lambda = ", lam)
    print("Old Position =", xi[k])
    print("New Position =", xi[k+1])
    
    print(abs(xi[k+1]-xi[k]))
    input(abs(xi[k+1]-xi[k]) < eps)
    
    if(all(abs(xi[k+1]-xi[k]) < eps)):
        break
    
    k+=1
print(xi[-1])

Starting Position =  [-4.99970372  5.00029628]
Objective Value =  1.7556322973177885e-07
Lambda Function =  0.5*(-0.000592559250481628*lamda - 4.99970372037476)**2 + (-0.000592559250481628*lamda - 4.99970372037476)*(-0.000592559250481628*lamda + 5.00029627962524) + 0.5*(-0.000592559250481628*lamda + 5.00029627962524)**2
*******************************
Executing Golden Ratio Search
*******************************
************************
[a,b] |b[k]-a[k]|
************************
0 1 1
0 0.618 0.618
0.236076 0.618 0.381924
0.382 0.618 0.236
0.472105032 0.618 0.145894968
0.472105032 0.562268122224 0.09016309022400004
0.472105032 0.527848 0.055742968000000004
0.49339884577599996 0.527848 0.03444915422400002
0.49339884577599996 0.5146884230864319 0.021289577310431962
0.49339884577599996 0.506547332465568 0.013148486689568084
0.498421567691415 0.506547332465568 0.008125764774153055
Lambda =  0.5024844500784915
Old Position = [-4.99970372  5.00029628]
New Position = [-5.00000147  4.99999853]

In [128]:
""" Convex Function Converging to the same location three times.

Function x^2 + y^2

Initial Position: [ 5 5]
0.0
0.0

Initial Position: [71  4]
[-4.32812136e-08 -2.43837823e-09]
-0.0000000432812136
-0.0000000243837823

Starting Position: [26 40]
-0.00000318973173
-0.00000490727959

"""